In [1]:
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

In [2]:
def warp_left_to_right(left_image_path, disparity_map_path, output_path):
    # 读取左影像和视差图
    left_image = cv2.imread(left_image_path)
    disparity_map = cv2.imread(disparity_map_path, cv2.IMREAD_GRAYSCALE)
    
    # 获取图像尺寸
    height, width = left_image.shape[:2]

    # 创建一个空的右影像
    right_image = np.zeros_like(left_image)
    
    # 将视差图的像素值从 uint8 转换为 float32
    disparity_map = disparity_map.astype(np.float32)
    
    # 遍历图像中的每一个像素
    for y in range(height):
        for x in range(width):
            disparity = disparity_map[y, x]
            if x - disparity >= 0:
                right_x = int(x - disparity)
                right_image[y, right_x] = left_image[y, x]
    
    # 保存右影像
    cv2.imwrite(output_path, right_image)

In [3]:
def warp_left_to_right_tensor(left_image_path, disparity_map_path, output_path):
    # 读取左影像和视差图
    left_image = Image.open(left_image_path)
    disparity_map = cv2.imread(disparity_map_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)

    # 将图像和视差图转换为tensor
    transform = transforms.ToTensor()
    left_image = transform(left_image)
    disparity_map = torch.from_numpy(disparity_map).unsqueeze(0)

    # 获取图像尺寸
    _, height, width = left_image.size()

    # 创建一个网格
    x = torch.linspace(-1, 1, width).view(1, -1).repeat(height, 1)
    y = torch.linspace(-1, 1, height).view(-1, 1).repeat(1, width)
    grid = torch.cat((x.unsqueeze(2), y.unsqueeze(2)), 2)
    grid = grid.unsqueeze(0)

    # 更新网格的x坐标
    grid[:, :, :, 0] = grid[:, :, :, 0] + 2 * disparity_map/width

    # 使用grid_sample函数进行重映射
    right_image = torch.nn.functional.grid_sample(left_image.unsqueeze(0), grid)

    # 保存右影像
    transforms.ToPILImage()(right_image.squeeze(0)).save(output_path)

In [4]:
left_image_path = 'vis/out_l.jpg'
disparity_map_path = 'vis/out_disp.jpg'
output_path = 'vis/warp_right.jpg'
warp_left_to_right_tensor(left_image_path, disparity_map_path, output_path)

c:\ProgramData\anaconda3\envs\torch\lib\site-packages\torch\nn\functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
